In [1]:
import os
os.getcwd() 

'C:\\Users\\Pc'

In [38]:
import urllib.error
import pandas as pd
from Bio import SeqIO, Entrez
from Bio.Seq import Seq
from openpyxl import workbook, load_workbook
from pydna.assembly import Assembly 
from pydna.dseq import Dseq
from pydna.dseqrecord import Dseqrecord

SsCC9311_VBPO_5_3 = "ATGACGGACCAACCTAAACTTACCGCGCAGCCTGTTCCTGAGGATGCTAACGCCCTGGCAGCCGGCCGAATCAACCCTCCTAATCAGGCTAACGGTGACGAGCAACGCTACGAGTCGGCCAACTACGCGATGTCTTTTACAAAAGGCCTGGACAATAACACCACCACCGGCCTGATTGAGCAGTCCGGTGACTTCGAAGCGTTTCGATCTGCTATTGACAATGGATTCGCTGAAGATTTCACCCGCAACGTCGCGGTACCGCCTGCGGAGCCGCCTCCTAAATTGGAAGCACCGACCGCGGGGACCGTATACGAGTTACAGGGTCCGGATCCGCAGGCAGTTACCATTCCCCCGGCCCCTGCGTTGTGTTCCGACGAACTGACATTTGAGATGGCCGAGGTCTACGAACTGGCCCTGCTGCGCGATCTGCCATTCAACGCCTTCGTCGCGGGGGGTGGATCCGCAGCGTTAGCGGACTCGACCGCCCCTCTCAATTCACTGGCTTACGCTCAGGATGGTTTTAACCGCCCTCCTCGCACCACCAACTCCTCTAATCAGCTGGATGCCCAGACCGTCTTTCGCGGCTCATCTCCGGGGGTTGACCAGGGTCCATACTTATCCCAGTTTATGCTGATCGGTAACGCGTCTCCATCGGAAGGCATCACCCCAGAGCAAGGTTTTATCAACTTCGGTGCCCAACCTATTGATCAGCCTGTACTGGAAGCGCGCCAGCAGGATGATTATATGATGAAATTGGATGACTTGAATCCTGTCCAGCAAGGGTACGAAGTGCGCGCGGATCCTTTTGACCCGTGCAAATCATCTGGTCCGGGCCAGGCATTCACGGGCCAACGCCCTTTCATTAACACGCCGCCTGATCTTGCCACATATGTGAACGTCGATGCCTTATATCAGGCATATCTCAATGCATGCCTGTTACTGCTTGGCAACGGCACTGCCTTCGACCCAGGTTTCGACCTGCTGTCCGGCGGCGGCGAAGGTCTGCTGAATGACCCAGCGGGTGGGCAAAAAGTCCCGCTGAACGCGGGCGGCTTCGCATTATTGGGCGGTCCGAACGTCTTATCTTTGGTTACCGAAGTTGCGACCCGAGGTCTGAAAGCGGTGCCTTACCAGAAATTCAATAACAACTTGCCTCTGCGCCCGGAAGCCCTGGCGGCGCGCATTGAAAAAGCGCAAGAAATTGAATCTCCTTTTCCGACGATTTGTGGCTGTTTTTCTGAAATGGCGTCAGATTTGCAGCAGGTGGTGGATCTGATCCGCAACAACAATCAGTCTCTAGCGGGCGAAGCGACCGCTCTGTTACCGATGGCATTTGCCGAGGGTTCTCCAATGAACCCTGCTTACGGTGCCGGAAATGCGACAGTGGCAGGTGCCTGCGTGACGATTCTGAAAGCGTTTTTCAACACATCCGCGTTGTTTGTGAAAATTAATGACGTGGCCGGCTTCAATTCAAAACAGAACATCCTCGCCCCTCTGAAATGCGGCGATTCAGTGGAAGCGGGTGCCTACCAGGAAACTGACTGCGGCAAACCTCTTGAATTTGAACCTTGCGGATCATTTAACTTGATTGAAGGCAAATATGCTACGTTTAAACCTGACGGGAAAACCAATCAGTCATGTTGCCCACTGACGCTGGAGGGTGAACTGAACAAGCTTGCAGCGAATATTTCTATTGGGCCTAATATGGCGGGTGTGAATTACTTCTCCGATTACTATGACTCTCTGCCTATGGGCGAGGAAATTGCCATAGGGATCTTGGAGGAACAGGCTCTGTGCTATAAAACCGACCCGTTCGTGCTCTCGGTTCCGACGTTTGACGGTGACGTTCGCCCTATTGGTCAACCT"
SsCC9311_VBPO_3_5 = "TACTGCCTGGTTGGATTTGAATGGCGCGTCGGACAAGGACTCCTACGATTGCGGGACCGTCGGCCGGCTTAGTTGGGAGGATTAGTCCGATTGCCACTGCTCGTTGCGATGCTCAGCCGGTTGATGCGCTACAGAAAATGTTTTCCGGACCTGTTATTGTGGTGGTGGCCGGACTAACTCGTCAGGCCACTGAAGCTTCGCAAAGCTAGACGATAACTGTTACCTAAGCGACTTCTAAAGTGGGCGTTGCAGCGCCATGGCGGACGCCTCGGCGGAGGATTTAACCTTCGTGGCTGGCGCCCCTGGCATATGCTCAATGTCCCAGGCCTAGGCGTCCGTCAATGGTAAGGGGGCCGGGGACGCAACACAAGGCTGCTTGACTGTAAACTCTACCGGCTCCAGATGCTTGACCGGGACGACGCGCTAGACGGTAAGTTGCGGAAGCAGCGCCCCCCACCTAGGCGTCGCAATCGCCTGAGCTGGCGGGGAGAGTTAAGTGACCGAATGCGAGTCCTACCAAAATTGGCGGGAGGAGCGTGGTGGTTGAGGAGATTAGTCGACCTACGGGTCTGGCAGAAAGCGCCGAGTAGAGGCCCCCAACTGGTCCCAGGTATGAATAGGGTCAAATACGACTAGCCATTGCGCAGAGGTAGCCTTCCGTAGTGGGGTCTCGTTCCAAAATAGTTGAAGCCACGGGTTGGATAACTAGTCGGACATGACCTTCGCGCGGTCGTCCTACTAATATACTACTTTAACCTACTGAACTTAGGACAGGTCGTTCCCATGCTTCACGCGCGCCTAGGAAAACTGGGCACGTTTAGTAGACCAGGCCCGGTCCGTAAGTGCCCGGTTGCGGGAAAGTAATTGTGCGGCGGACTAGAACGGTGTATACACTTGCAGCTACGGAATATAGTCCGTATAGAGTTACGTACGGACAATGACGAACCGTTGCCGTGACGGAAGCTGGGTCCAAAGCTGGACGACAGGCCGCCGCCGCTTCCAGACGACTTACTGGGTCGCCCACCCGTTTTTCAGGGCGACTTGCGCCCGCCGAAGCGTAATAACCCGCCAGGCTTGCAGAATAGAAACCAATGGCTTCAACGCTGGGCTCCAGACTTTCGCCACGGAATGGTCTTTAAGTTATTGTTGAACGGAGACGCGGGCCTTCGGGACCGCCGCGCGTAACTTTTTCGCGTTCTTTAACTTAGAGGAAAAGGCTGCTAAACACCGACAAAAAGACTTTACCGCAGTCTAAACGTCGTCCACCACCTAGACTAGGCGTTGTTGTTAGTCAGAGATCGCCCGCTTCGCTGGCGAGACAATGGCTACCGTAAACGGCTCCCAAGAGGTTACTTGGGACGAATGCCACGGCCTTTACGCTGTCACCGTCCACGGACGCACTGCTAAGACTTTCGCAAAAAGTTGTGTAGGCGCAACAAACACTTTTAATTACTGCACCGGCCGAAGTTAAGTTTTGTCTTGTAGGAGCGGGGAGACTTTACGCCGCTAAGTCACCTTCGCCCACGGATGGTCCTTTGACTGACGCCGTTTGGAGAACTTAAACTTGGAACGCCTAGTAAATTGAACTAACTTCCGTTTATACGATGCAAATTTGGACTGCCCTTTTGGTTAGTCAGTACAACGGGTGACTGCGACCTCCCACTTGACTTGTTCGAACGTCGCTTATAAAGATAACCCGGATTATACCGCCCACACTTAATGAAGAGGCTAATGATACTGAGAGACGGATACCCGCTCCTTTAACGGTATCCCTAGAACCTCCTTGTCCGAGACACGATATTTTGGCTGGGCAAGCACGAGAGCCAAGGCTGCAAACTGCCACTGCAAGCGGGATAACCAGTTGGA"

SsWH8020_VBPO_5_3 = "ATGACAGACCAGCCTAAACTGACCGCCCAGCCTGTTCGCGAAGACGCCAATGCTCTGGCGGCTGGCCGCGTGCAACCGCAGAACCAAGCCAACGGGGACGAGCAGCGCTACGAGTCCGCATCGTACCCGATGTCGTTCACGAAAGGTCTGGACAATAATACCACGACCGGCCTGGTTGAACAATCCGGCGATTTCGAAGCGTTTCCTTCAGCCATCGACAACGGCTTTGCCGAAGATTTTACTCGCCAGGTTCCTGTTCCAAATGCTGAGCCGCGGCGGAAATTGGAAGCCCCAACCGCTGGGACAGTGTATGAACTGCAGGGACCGGACCCGCAGGCTGTCACTATCCCTCCGGCCCCGGCACTGTGTTCAGATGAACTGACGTTCGAGATCGCAGAAGTCTACGAACTTGCTCTTCTGCGCGATTTACCCTTCAACGCTTTCGTGGCCGGTGGTGGCTCTGCTGCTTTGGCAGACTCAACGGCTCGGCTGAATAACCTGGCGTACGCCCAGGACGGTTTCAACTCTCCTCCACGCAAGACTAACTCTTCAAAGCAGCTGGATGCACAGACAGTGTTTCGGGGTTCTTCTCCTGGTGTAGAACCTGGTCCATACCTGTCGCAGTTTATGCTGATCGGTAACGCGTCTCCGTCGGAAGGCATTACGCCAGATCAGGGCTTCATCAACTTTGGTGCGCAGCCTATCGATCAGCCTGTTCTGGAGGCACCTCAGCAAGACGATTATATGATGAAGTTGGACGACTTGAACCGCGTGGAGCAGGGATATGAGGTTCGCGGCGACCCTTTCGATCCGTGCAAGTCATCCGGTCCGGGCCAGGCGTTTACGGGCCAGCCTCCTTTTATTAACACCCCGCCTGACTTAGCCACTTACGTTAATGTGGACGCGCTGTATCAAGCTTACCTGAACGCGTGCTTACTGTTACTTGGCAATGGTACCCCGTTCGACCCGGGCTTCGATCTGCTGTCTGGCGGCGGCGAAGGCCTGCTAAACGATCCGGCATCCGGTCAAAAGGTGCCGCTGAATGCCGGCGGGTTCGCATTGTTGGGGGGGCCTAACGTACTGTCACTAGTGACCGAAGTGGCAACCCCTGGTCTGAAAGCTGTTCCTTACCAAAAATTCAACAACAACTTGCGCCTGCGCCCGGAAGCACTGGCAGCACGCATTGAGAAAGCGCAGGAGATTGAATCACCTTTTCCTGAAATATGTGGCTGTTTCTCGGAGATGGCTTCAGACCTACAACAAACAGTAGACCTGATTCCTAATAATAACCAATCACTGGCTGGCGATGCGACCGCTTTGCTGCCTATGGCCTTTGCGGAAGGTTCACCGATGAATCCCGCATATGGGGCTGGTAACGCGACGGTTGCGGGCGCCTGTGTCACCATCCTGAAAGCATTCTTCAACACGTCCGCTTTATTTGTAAAAATCAATGACTTGGCAGGCTTTTATTCCAAGCAGAACATTCTCGATCCTCTGAAATGCGGAGACTCTGTCGAAGCAGGTGCGTATCAAGTAACGGATTGTGGTAAACGCCTTGAGTTCGAACGCTGTGGCTCTTTTAACTTGATTGAAGGTAAAGATGCGACCTTTAAACCGGATGGCACTACTAACAAGTCATGCTGCCCCCTTACACTTGAAGGTGAGTTGAATAAGCTGGCGGCAAATATTTCTATCGGTCGCAACATGGCAGGGGTAAACTATTTTTCGGACTATTACGACTCTCTGCCTATGGGCGAGGAGATCGCGATCGGTATCCTCGAGGAGCAGGCGCTTTGCTACAAGACCGACCCCTTTGTTCTGTCCGTCCCGACATTTGACGGTGACGTCCGCCCTATCGGCCAGCCT"
SsWH8020_VBPO_3_5 = "TACTGTCTGGTCGGATTTGACTGGCGGGTCGGACAAGCGCTTCTGCGGTTACGAGACCGCCGACCGGCGCACGTTGGCGTCTTGGTTCGGTTGCCCCTGCTCGTCGCGATGCTCAGGCGTAGCATGGGCTACAGCAAGTGCTTTCCAGACCTGTTATTATGGTGCTGGCCGGACCAACTTGTTAGGCCGCTAAAGCTTCGCAAAGGAAGTCGGTAGCTGTTGCCGAAACGGCTTCTAAAATGAGCGGTCCAAGGACAAGGTTTACGACTCGGCGCCGCCTTTAACCTTCGGGGTTGGCGACCCTGTCACATACTTGACGTCCCTGGCCTGGGCGTCCGACAGTGATAGGGAGGCCGGGGCCGTGACACAAGTCTACTTGACTGCAAGCTCTAGCGTCTTCAGATGCTTGAACGAGAAGACGCGCTAAATGGGAAGTTGCGAAAGCACCGGCCACCACCGAGACGACGAAACCGTCTGAGTTGCCGAGCCGACTTATTGGACCGCATGCGGGTCCTGCCAAAGTTGAGAGGAGGTGCGTTCTGATTGAGAAGTTTCGTCGACCTACGTGTCTGTCACAAAGCCCCAAGAAGAGGACCACATCTTGGACCAGGTATGGACAGCGTCAAATACGACTAGCCATTGCGCAGAGGCAGCCTTCCGTAATGCGGTCTAGTCCCGAAGTAGTTGAAACCACGCGTCGGATAGCTAGTCGGACAAGACCTCCGTGGAGTCGTTCTGCTAATATACTACTTCAACCTGCTGAACTTGGCGCACCTCGTCCCTATACTCCAAGCGCCGCTGGGAAAGCTAGGCACGTTCAGTAGGCCAGGCCCGGTCCGCAAATGCCCGGTCGGAGGAAAATAATTGTGGGGCGGACTGAATCGGTGAATGCAATTACACCTGCGCGACATAGTTCGAATGGACTTGCGCACGAATGACAATGAACCGTTACCATGGGGCAAGCTGGGCCCGAAGCTAGACGACAGACCGCCGCCGCTTCCGGACGATTTGCTAGGCCGTAGGCCAGTTTTCCACGGCGACTTACGGCCGCCCAAGCGTAACAACCCCCCCGGATTGCATGACAGTGATCACTGGCTTCACCGTTGGGGACCAGACTTTCGACAAGGAATGGTTTTTAAGTTGTTGTTGAACGCGGACGCGGGCCTTCGTGACCGTCGTGCGTAACTCTTTCGCGTCCTCTAACTTAGTGGAAAAGGACTTTATACACCGACAAAGAGCCTCTACCGAAGTCTGGATGTTGTTTGTCATCTGGACTAAGGATTATTATTGGTTAGTGACCGACCGCTACGCTGGCGAAACGACGGATACCGGAAACGCCTTCCAAGTGGCTACTTAGGGCGTATACCCCGACCATTGCGCTGCCAACGCCCGCGGACACAGTGGTAGGACTTTCGTAAGAAGTTGTGCAGGCGAAATAAACATTTTTAGTTACTGAACCGTCCGAAAATAAGGTTCGTCTTGTAAGAGCTAGGAGACTTTACGCCTCTGAGACAGCTTCGTCCACGCATAGTTCATTGCCTAACACCATTTGCGGAACTCAAGCTTGCGACACCGAGAAAATTGAACTAACTTCCATTTCTACGCTGGAAATTTGGCCTACCGTGATGATTGTTCAGTACGACGGGGGAATGTGAACTTCCACTCAACTTATTCGACCGCCGTTTATAAAGATAGCCAGCGTTGTACCGTCCCCATTTGATAAAAAGCCTGATAATGCTGAGAGACGGATACCCGCTCCTCTAGCGCTAGCCATAGGAGCTCCTCGTCCGCGAAACGATGTTCTGGCTGGGGAAACAAGACAGGCAGGGCTGTAAACTGCCACTGCAGGCGGGATAGCCGGTCGGA"

def reverse(string):
    string = string[::-1]
    return string

In [55]:
SsCC9311_VBPO = Dseqrecord(SsCC9311_VBPO_5_3, linear=True, circular=False)
SsWH8020_VBPO = Dseqrecord(SsWH8020_VBPO_5_3, linear=True, circular=False)

In [58]:
SsCC9311_VBPO.seq

Dseq(-1866)
ATGA..ACCT
TACT..TGGA

In [59]:
SsWH8020_VBPO.seq

Dseq(-1866)
ATGA..GCCT
TACT..CGGA

In [60]:
plasmid_5_3 = Seq("ATCCGGATATAGTTCCTCCTTTCAGCAAAAAACCCCTCAAGACCCGTTTAGAGGCCCCAAGGGGTTATGCTAGTTATTGCTCAGCGGTGGCAGCAGCCAACTCAGCTTCCTTTCGGGCTTTGTTTAGCAGCCTAGGTATTAATCAATTAGTGGTGGTGGTGGTGGTGGTGGTGCTCGAGTGCGGCCGCAAGCTTGTCGACGGAGCTCGAATTCGGATCCGATATCGCCATGGTTGAGGAGAAGCCCGGGCTCTTGTCGTCGTCATCGGTACCCAGATCTGGGCTGTCCATGTGCTGGCGTTCGAATTTAGCAGCAGCGGTTTCTTTCATACCAATTGCAGTACTACCGCGTGGCACCAGACCCGCGGAGTGATGGTGATGGTGATGACCAGAACCACTAGTTGATCCTTTTTTCTCGCTTAAGTATTTCACTGTATCAGCATACTGCTGAACAAAAACATCCATATTGCTGGTATCCATACCCTGCGGATTCAGCTGATATTTACCGTTAACAAACATCGCCGGAACGCCACGCAATTGCACGTCAGCTGCAGCTTTTTCCTGCTGAGCGACCAGAGATTTCACCACGAAGCTGTTCCACGCCGCGTCGTACTCTTCACCTTTAATACCTGCGTTGATAAATACATCGCGGATATCAGAAGCAGAACGAATGGTCTGGGTTTTCTGTACGCCTTCAAACAGCGGAACAGTCACTTTGTCTTCCACGCCCAGCGCCATCGCCACAGCCCATGCCTGAGTCAGATCTTTGCCCAGGTCACCACCCATGAAGTTGACGTGGTATTTAGTCATCTTCACGCCTTCCGGCAGTTTTTTCTTCACATTATCAGAAATATGCAGAACTTCTTCAAACTGATAGCAGTGCGGGCAGAAGAAAGAGAAAAACTCCAGCACTTGCGGCGCGCCAGCTACCGGTTTTTCCAGGGTAGTGTACTGTTTACCATCTTCATACTGCGCCGCCGATGCGCTAAACGCTAAAACTAAACCAGCCAGCGCCAGCCAAATCTTTTTCATATGTATATCTCCTTCTTAAAGTTAAACAAAATTATTTCTAGAGGGGAATTGTTATCCGCTCACAATTCCCCTATAGTGAGTCGTATTAATTTCGCGGGATCGAGATCGATCTCGATCCTCTACGCCGGACGCATCGTGGCCGGCATCACCGGCGCCACAGGTGCGGTTGCTGGCGCCTATATCGCCGACATCACCGATGGGGAAGATCGGGCTCGCCACTTCGGGCTCATGAGCGCTTGTTTCGGCGTGGGTATGGTGGCAGGCCCCGTGGCCGGGGGACTGTTGGGCGCCATCTCCTTGCATGCACCATTCCTTGCGGCGGCGGTGCTCAACGGCCTCAACCTACTACTGGGCTGCTTCCTAATGCAGGAGTCGCATAAGGGAGAGCGTCGAGATCCCGGACACCATCGAATGGCGCAAAACCTTTCGCGGTATGGCATGATAGCGCCCGGAAGAGAGTCAATTCAGGGTGGTGAATGTGAAACCAGTAACGTTATACGATGTCGCAGAGTATGCCGGTGTCTCTTATCAGACCGTTTCCCGCGTGGTGAACCAGGCCAGCCACGTTTCTGCGAAAACGCGGGAAAAAGTGGAAGCGGCGATGGCGGAGCTGAATTACATTCCCAACCGCGTGGCACAACAACTGGCGGGCAAACAGTCGTTGCTGATTGGCGTTGCCACCTCCAGTCTGGCCCTGCACGCGCCGTCGCAAATTGTCGCGGCGATTAAATCTCGCGCCGATCAACTGGGTGCCAGCGTGGTGGTGTCGATGGTAGAACGAAGCGGCGTCGAAGCCTGTAAAGCGGCGGTGCACAATCTTCTCGCGCAACGCGTCAGTGGGCTGATCATTAACTATCCGCTGGATGACCAGGATGCCATTGCTGTGGAAGCTGCCTGCACTAATGTTCCGGCGTTATTTCTTGATGTCTCTGACCAGACACCCATCAACAGTATTATTTTCTCCCATGAAGACGGTACGCGACTGGGCGTGGAGCATCTGGTCGCATTGGGTCACCAGCAAATCGCGCTGTTAGCGGGCCCATTAAGTTCTGTCTCGGCGCGTCTGCGTCTGGCTGGCTGGCATAAATATCTCACTCGCAATCAAATTCAGCCGATAGCGGAACGGGAAGGCGACTGGAGTGCCATGTCCGGTTTTCAACAAACCATGCAAATGCTGAATGAGGGCATCGTTCCCACTGCGATGCTGGTTGCCAACGATCAGATGGCGCTGGGCGCAATGCGCGCCATTACCGAGTCCGGGCTGCGCGTTGGTGCGGACATCTCGGTAGTGGGATACGACGATACCGAAGACAGCTCATGTTATATCCCGCCGTTAACCACCATCAAACAGGATTTTCGCCTGCTGGGGCAAACCAGCGTGGACCGCTTGCTGCAACTCTCTCAGGGCCAGGCGGTGAAGGGCAATCAGCTGTTGCCCGTCTCACTGGTGAAAAGAAAAACCACCCTGGCGCCCAATACGCAAACCGCCTCTCCCCGCGCGTTGGCCGATTCATTAATGCAGCTGGCACGACAGGTTTCCCGACTGGAAAGCGGGCAGTGAGCGCAACGCAATTAATGTAAGTTAGCTCACTCATTAGGCACCGGGATCTCGACCGATGCCCTTGAGAGCCTTCAACCCAGTCAGCTCCTTCCGGTGGGCGCGGGGCATGACTATCGTCGCCGCACTTATGACTGTCTTCTTTATCATGCAACTCGTAGGACAGGTGCCGGCAGCGCTCTGGGTCATTTTCGGCGAGGACCGCTTTCGCTGGAGCGCGACGATGATCGGCCTGTCGCTTGCGGTATTCGGAATCTTGCACGCCCTCGCTCAAGCCTTCGTCACTGGTCCCGCCACCAAACGTTTCGGCGAGAAGCAGGCCATTATCGCCGGCATGGCGGCCCCACGGGTGCGCATGATCGTGCTCCTGTCGTTGAGGACCCGGCTAGGCTGGCGGGGTTGCCTTACTGGTTAGCAGAATGAATCACCGATACGCGAGCGAACGTGAAGCGACTGCTGCTGCAAAACGTCTGCGACCTGAGCAACAACATGAATGGTCTTCGGTTTCCGTGTTTCGTAAAGTCTGGAAACGCGGAAGTCAGCGCCCTGCACCATTATGTTCCGGATCTGCATCGCAGGATGCTGCTGGCTACCCTGTGGAACACCTACATCTGTATTAACGAAGCGCTGGCATTGACCCTGAGTGATTTTTCTCTGGTCCCGCCGCATCCATACCGCCAGTTGTTTACCCTCACAACGTTCCAGTAACCGGGCATGTTCATCATCAGTAACCCGTATCGTGAGCATCCTCTCTCGTTTCATCGGTATCATTACCCCCATGAACAGAAATCCCCCTTACACGGAGGCATCAGTGACCAAACAGGAAAAAACCGCCCTTAACATGGCCCGCTTTATCAGAAGCCAGACATTAACGCTTCTGGAGAAACTCAACGAGCTGGACGCGGATGAACAGGCAGACATCTGTGAATCGCTTCACGACCACGCTGATGAGCTTTACCGCAGCTGCCTCGCGCGTTTCGGTGATGACGGTGAAAACCTCTGACACATGCAGCTCCCGGAGACGGTCACAGCTTGTCTGTAAGCGGATGCCGGGAGCAGACAAGCCCGTCAGGGCGCGTCAGCGGGTGTTGGCGGGTGTCGGGGCGCAGCCATGACCCAGTCACGTAGCGATAGCGGAGTGTATACTGGCTTAACTATGCGGCATCAGAGCAGATTGTACTGAGAGTGCACCATATATGCGGTGTGAAATACCGCACAGATGCGTAAGGAGAAAATACCGCATCAGGCGCTCTTCCGCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAGGCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGTAAAAAGGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCATAGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGGACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGAAGATCCTTTGATCTTTTCTACGGGGTCTGACGCTCAGTGGAACGAAAACTCACGTTAAGGGATTTTGGTCATGAACAATAAAACTGTCTGCTTACATAAACAGTAATACAAGGGGTGTTATGAGCCATATTCAACGGGAAACGTCTTGCTCTAGGCCGCGATTAAATTCCAACATGGATGCTGATTTATATGGGTATAAATGGGCTCGCGATAATGTCGGGCAATCAGGTGCGACAATCTATCGATTGTATGGGAAGCCCGATGCGCCAGAGTTGTTTCTGAAACATGGCAAAGGTAGCGTTGCCAATGATGTTACAGATGAGATGGTCAGACTAAACTGGCTGACGGAATTTATGCCTCTTCCGACCATCAAGCATTTTATCCGTACTCCTGATGATGCATGGTTACTCACCACTGCGATCCCCGGGAAAACAGCATTCCAGGTATTAGAAGAATATCCTGATTCAGGTGAAAATATTGTTGATGCGCTGGCAGTGTTCCTGCGCCGGTTGCATTCGATTCCTGTTTGTAATTGTCCTTTTAACAGCGATCGCGTATTTCGTCTCGCTCAGGCGCAATCACGAATGAATAACGGTTTGGTTGATGCGAGTGATTTTGATGACGAGCGTAATGGCTGGCCTGTTGAACAAGTCTGGAAAGAAATGCATAAACTTTTGCCATTCTCACCGGATTCAGTCGTCACTCATGGTGATTTCTCACTTGATAACCTTATTTTTGACGAGGGGAAATTAATAGGTTGTATTGATGTTGGACGAGTCGGAATCGCAGACCGATACCAGGATCTTGCCATCCTATGGAACTGCCTCGGTGAGTTTTCTCCTTCATTACAGAAACGGCTTTTTCAAAAATATGGTATTGATAATCCTGATATGAATAAATTGCAGTTTCATTTGATGCTCGATGAGTTTTTCTAAGAATTAATTCATGAGCGGATACATATTTGAATGTATTTAGAAAAATAAACAAATAGGGGTTCCGCGCACATTTCCCCGAAAAGTGCCACCTGAAATTGTAAACGTTAATATTTTGTTAAAATTCGCGTTAAATTTTTGTTAAATCAGCTCATTTTTTAACCAATAGGCCGAAATCGGCAAAATCCCTTATAAATCAAAAGAATAGACCGAGATAGGGTTGAGTGTTGTTCCAGTTTGGAACAAGAGTCCACTATTAAAGAACGTGGACTCCAACGTCAAAGGGCGAAAAACCGTCTATCAGGGCGATGGCCCACTACGTGAACCATCACCCTAATCAAGTTTTTTGGGGTCGAGGTGCCGTAAAGCACTAAATCGGAACCCTAAAGGGAGCCCCCGATTTAGAGCTTGACGGGGAAAGCCGGCGAACGTGGCGAGAAAGGAAGGGAAGAAAGCGAAAGGAGCGGGCGCTAGGGCGCTGGCAAGTGTAGCGGTCACGCTGCGCGTAACCACCACACCCGCCGCGCTTAATGCGCCGCTACAGGGCGCGTCCCATTCGCCA")
plasmid_3_5 = plasmid_5_3.reverse_complement()

plasmid = Dseqrecord(str(plasmid_5_3), linear=False, circular=True)

In [61]:
plasmid

Dseqrecord(o6106)

In [62]:
from Bio.Restriction import *

In [63]:
linear_plasmid = plasmid.linearize(XbaI)

In [64]:
linear_plasmid

Dseqrecord(-6110)

In [82]:
linear_plasmid.seq

Dseq(-6110)
CTAGAGGG..ATTT    
    TCCC..TAAAGATC

In [83]:
x = Assembly((linear_plasmid, SsWH8020_VBPO, SsCC9311_VBPO), limit = 15)

In [84]:
x

Assembly
fragments..: 6110bp 1866bp 1866bp
limit(bp)..: 15
G.nodes....: 6
algorithm..: common_sub_strings

In [85]:
x.assemble_circular()

[]

In [87]:
x.detailed_figure()

AttributeError: 'Assembly' object has no attribute 'detailed_figure'